<a href="https://colab.research.google.com/github/AgileDataScientist/absent-data-hospital-wait-time/blob/main/Hospital_Wait_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install plotly
!pip install xlrd
!pip install pyjanitor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 147 kB 7.4 MB/s 


In [28]:
# Standard imports
import pandas as pd
import numpy as np

# pyjanitor import
import janitor
from janitor import clean_names, remove_empty

# Mount Google drive
from google.colab import drive
drive.mount('/content/myDrive')

Drive already mounted at /content/myDrive; to attempt to forcibly remount, call drive.mount("/content/myDrive", force_remount=True).


In [24]:
hospital_df = pd.read_excel('/content/myDrive/MyDrive/Data Analyst Portfolio/Data Analysis Hospital Wait Time/hospital_data_sample.xlsx', sheet_name='Sheet1')
hospital_df.head()

,Date,Medication Revenue,Lab Cost,Consultation Revenue,Doctor Type,Financial Class,Patient Type,Entry Time,Post-Consultation Time,Completion Time,Patient ID
0,2019-11-04,1183.22,10,20.17,ANCHOR,HMO,OUTPATIENT,08:35:45,09:17:54,09:29:46,C10001
1,2019-11-06,738.48,$-,15,ANCHOR,INSURANCE,OUTPATIENT,19:19:16,21:02:36,21:24:07,C10002
2,2019-11-02,660,$-,21.17,ANCHOR,HMO,OUTPATIENT,10:46:52,11:56:25,12:06:28,C10003
3,2019-11-06,600,$-,$-,ANCHOR,MEDICARE,OUTPATIENT,09:38:34,10:55:50,10:58:02,C10004
4,2019-11-01,591.6,$-,12,ANCHOR,INSURANCE,OUTPATIENT,11:16:21,12:06:49,12:06:54,C10005


In [25]:
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    29998 non-null  datetime64[ns]
 1    Medication Revenue     29998 non-null  object        
 2     Lab Cost              29998 non-null  object        
 3    Consultation Revenue   29998 non-null  object        
 4   Doctor Type             29998 non-null  object        
 5   Financial Class         29998 non-null  object        
 6   Patient Type            29998 non-null  object        
 7   Entry Time              29998 non-null  object        
 8   Post-Consultation Time  29998 non-null  object        
 9   Completion Time         29998 non-null  object        
 10  Patient ID              29998 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 2.5+ MB


In [33]:
hospital_df = clean_names(hospital_df)
hospital_df.head()

,date,_medication_revenue_,_lab_cost_,_consultation_revenue_,doctor_type,financial_class,patient_type,entry_time,post_consultation_time,completion_time,patient_id
0,2019-11-04,1183.22,10,20.17,ANCHOR,HMO,OUTPATIENT,08:35:45,09:17:54,09:29:46,C10001
1,2019-11-06,738.48,$-,15,ANCHOR,INSURANCE,OUTPATIENT,19:19:16,21:02:36,21:24:07,C10002
2,2019-11-02,660,$-,21.17,ANCHOR,HMO,OUTPATIENT,10:46:52,11:56:25,12:06:28,C10003
3,2019-11-06,600,$-,$-,ANCHOR,MEDICARE,OUTPATIENT,09:38:34,10:55:50,10:58:02,C10004
4,2019-11-01,591.6,$-,12,ANCHOR,INSURANCE,OUTPATIENT,11:16:21,12:06:49,12:06:54,C10005


In [66]:
from datetime import datetime

# Convert time to time format
hospital_df["entry_time"] = pd.to_datetime(hospital_df["entry_time"].astype(str))
hospital_df["post_consultation_time"] = pd.to_datetime(hospital_df["post_consultation_time"].astype(str))
hospital_df["completion_time"] = pd.to_datetime(hospital_df["completion_time"].astype(str))

In [78]:
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   date                    29998 non-null  datetime64[ns] 
 1   _medication_revenue_    29998 non-null  object         
 2   _lab_cost_              29998 non-null  object         
 3   _consultation_revenue_  29998 non-null  object         
 4   doctor_type             29998 non-null  object         
 5   financial_class         29998 non-null  object         
 6   patient_type            29998 non-null  object         
 7   entry_time              29998 non-null  datetime64[ns] 
 8   post_consultation_time  29998 non-null  datetime64[ns] 
 9   completion_time         29998 non-null  datetime64[ns] 
 10  patient_id              29998 non-null  object         
 11  wait_time               29998 non-null  timedelta64[ns]
dtypes: datetime64[ns](4), object(7),

In [136]:
def time_difference(time1, time2):
  difference = abs(pd.to_timedelta(time1 - time2))
  return difference / np.timedelta64(1, 's') / 60

In [139]:
hospital_df["wait_time_min"] = time_difference(hospital_df["entry_time"], hospital_df["post_consultation_time"])
hospital_df["consultation_time_min"] = time_difference(hospital_df["entry_time"], hospital_df["completion_time"])
hospital_df["consultation_time_min"]

0         54.016667
1        124.850000
2         79.600000
3         79.466667
4         50.550000
            ...    
29993     14.300000
29994     20.333333
29995     14.800000
29996     11.666667
29997     17.116667
Name: consultation_time_min, Length: 29998, dtype: float64

In [74]:
date_format_str = '%d/%m/%Y %H:%M:%S.%f'
start = datetime.strptime(date_1, date_format_str)
end =   datetime.strptime(date_2, date_format_str)
# Get interval between two datetimes as timedelta object
diff = end - start
# Get the interval in minutes
diff_in_minutes = diff.total_seconds() / 60

NameError: ignored